# tourapi

In [ ]:
## shopping 

In [1]:
import pandas as pd
tourapi = pd.read_excel('/home/hwang/projects/yanolja/data/dataset/tourAPI/shopping.xls')

In [2]:
tourapi['source'] = 'tourAPI'

In [3]:
tourapi['category'] = "shopping"

In [4]:
tourapi.head(2)

,명칭,우편번호,관리자,전화번호,주소,위도,경도,개요,문의 및 안내,규모,...,매장안내,주차시설,문화센터 바로가기,유모차 대여 여부,애완동물 동반 가능 여부,신용카드 가능 여부,화장실,상세정보,source,category
0,동문재래시장,63192,동문재래시장,NaN,제주특별자치도 제주시 관덕로14길 20,33.511562,126.526059,동문재래시장은 오랜 역사를 지닌 전통 재래시장으로 제주도를 대표하는 시장 중 하나...,064-752-3001,"- 점포수 300여 개 점포<br />\n- 면적 : 17,299㎡<br />\n-...",...,NaN,가능,NaN,없음,NaN,없음,있음,NaN,tourAPI,shopping
1,광장시장 한복매장,3195,NaN,NaN,서울특별시 종로구 창경궁로 88(예지동),37.570065,126.998908,현대화를 시도 하고 있는 다른 재래시장들과는 달리 광장시장은 오래된 역사와 한국 전...,광장시장 상인총연합회 02-2272-0967,NaN,...,"1층 : 식자재, 이불, 승복지, 원단, 잡화 등<br/>\n2층 : 한복, 포목,...",NaN,NaN,불가,NaN,가능,있음,NaN,tourAPI,shopping


In [5]:
tourapi.rename(columns= {'명칭': 'place_name', 
                                   '주소':'address', 
                                   '위도': 'latitude', 
                                   '경도': 'longitude', 
                                   '영업시간': 'available_time', ## 영업시간
                                   '주차시설': 'parking', 
                                   '개요': 'description', 
                                   '우편번호': 'zip_code', 
                                   '관리자': 'manager',
                                   '전화번호': 'phone',
                                   '문의 및 안내': 'guidance', 
                                   '규모': 'scale',
                                   '쉬는날': 'day_off', 
                                   '판매품목': 'items',
                                   '품목별 가격': 'prices',
                                   '장서는 날': 'market_day',
                                   '개장일': 'opening_date',
                                   '매장안내': 'store_information',
                                   '문화센터 바로가기': 'url',
                                   '화장실': 'toilet',
                                   '유모차 대여 여부': 'stroller', 
                                   '애완동물 동반 가능 여부': 'pet',
                                   '신용카드 가능 여부': 'credit_card',
                                   '상세정보': 'details'}, inplace=True)

tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'parking', 'description', 'zip_code', 'manager', 'phone', 'guidance', 'scale', 'day_off', 
                            'items', 'prices', 'market_day', 'opening_date', 'store_information', 'url', 'toilet', 'stroller', 'pet', 'credit_card', 'details']]

In [6]:
tourapi.isnull().sum() / len(tourapi)

source               0.000000
category             0.000000
place_name           0.000000
address              0.000000
latitude             0.000000
longitude            0.000000
available_time       0.016576
parking              0.042084
description          0.000298
zip_code             0.001191
manager              0.955533
phone                0.939950
guidance             0.050620
scale                0.954442
day_off              0.433251
items                0.007444
prices               0.996526
market_day           0.117022
opening_date         0.946402
store_information    0.106203
url                  0.996526
toilet               0.168139
stroller             0.869280
pet                  0.912655
credit_card          0.012903
details              0.971712
dtype: float64

In [7]:
tourapi.isnull().sum()

source                   0
category                 0
place_name               0
address                  0
latitude                 0
longitude                0
available_time         167
parking                424
description              3
zip_code                12
manager               9627
phone                 9470
guidance               510
scale                 9616
day_off               4365
items                   75
prices               10040
market_day            1179
opening_date          9535
store_information     1070
url                  10040
toilet                1694
stroller              8758
pet                   9195
credit_card            130
details               9790
dtype: int64

In [8]:
len(tourapi)

10075

In [ ]:
import openai
import pandas as pd
import re
import json
import os
from tqdm import tqdm
from dotenv import load_dotenv

if not load_dotenv(dotenv_path="/home/hwang/projects/yanolja/src/.env"):
    print(".env 파일을 로드하지 못했습니다. 경로를 확인하세요.")
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

df = tourapi[5000:]

def preprocess_text(text):
    if pd.isna(text) or text.strip() == "":
        return "정보 없음"
    text = re.sub(r"<[^>]+>", " ", text)  
    text = re.sub(r"\s+", " ", text).strip()  
    return text

df["day_off"] = df["day_off"].apply(preprocess_text)
df["available_time"] = df["available_time"].apply(preprocess_text)

def transform_to_standard_format(day_off, available_time):
    day_off = preprocess_text(str(day_off))
    available_time = preprocess_text(str(available_time))
    
    prompt = f"""

    **입력 데이터**:
    - 영업시간(available_time): {available_time}
    - 휴무일(day_off): {day_off}

    입력 데이터에 대해서 이해한 바를 창조하지 말고 사실에 기반하여 서술하세요.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=500
    )
    content = response['choices'][0]['message']['content']
    return content


results = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    transformed = transform_to_standard_format(row["day_off"], row["available_time"])
    results.append({"row_index": index, "transformed_data": transformed})

output_file = "transformed_tourapi_shopping_available_time_results_v1_3.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"변환된 결과가 {output_file} 파일에 저장되었습니다.")


/tmp/ipykernel_3978957/3122279911.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["day_off"] = df["day_off"].apply(preprocess_text)
/tmp/ipykernel_3978957/3122279911.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["available_time"] = df["available_time"].apply(preprocess_text)
Processing rows:   0%|          | 4/5075 [00:07<2:22:25,  1.69s/it]

In [10]:
import json
import openai

import os
from dotenv import load_dotenv

from tqdm import tqdm

load_dotenv(dotenv_path="/home/hwang/projects/yanolja/src/.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

input_file = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_shopping_available_time_results_v1_3.json"
output_file = "transformed_tourapi_shopping_available_time_results_v2_3.json"

def transform_data_with_gpt(input_text):
    system_prompt = (
        "You are an assistant that transforms text about business hours and holidays into a structured JSON format. "
        "The JSON must strictly adhere to the following rules and reflect only the information explicitly provided in the input text. "
        "If the input specifies different 'entry closing time' (e.g., last entry allowed) and 'closing time' (e.g., business hours end), these must be clearly separated:\n\n"
        "Rules for transformation:\n"
        "1. The 'hours' key should only include the official business hours for each day (Monday to Sunday).\n"
        "   - If no business hours are provided for a specific day, leave that day as an empty list (e.g., \"Monday\": []).\n"
        "   - 'Closing time' refers to the time when the business officially stops operating and should be used in the 'hours' key.\n\n"
        "2. The 'special_conditions' key should include:\n"
        "   - 'entry closing time' (last entry allowed) as a separate condition with 'type: \"entry_closing_time\"' if explicitly stated.\n"
        "   - Any holidays, seasonal hours, or other exceptions.\n"
        "   - Each entry must have a 'type' and 'details'.\n\n"
        "3. If conflicting or ambiguous information is present in the input, separate the conflicting details into individual entries in 'special_conditions'.\n\n"
        "4. Do not infer or assume any information. Only use the data explicitly provided in the input text.\n"
        "   - If any information is missing or incomplete, leave the corresponding field empty (e.g., `[]`, `{}`).\n"
        "   - Only add 'details: \"Information not provided\"' if explicitly mentioned in the input text."
    )

    user_prompt = (
        f"Transform the following text into the required JSON format. "
        "Ensure that the output strictly adheres to the rules provided and includes only the information explicitly stated in the input text. "
        "If entry closing time (last entry allowed) and closing time (business end) are mentioned, clearly separate them. "
        "Add entry closing time as a 'special_condition' with 'type: \"entry_closing_time\"'. "
        "If information is missing, ambiguous, or incomplete, leave fields empty (e.g., `[]`, `{}`) and do not make assumptions:\n\n"
        f"Text: {input_text}\n\n"
        "Output example:\n"
        "{\n"
        '  "hours": {\n'
        '    "Monday": [],\n'
        '    "Tuesday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Wednesday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Thursday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Friday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Saturday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Sunday": [{"start": "10:00", "end": "18:00"}]\n'
        "  },\n"
        '  "special_conditions": [\n'
        '    {"type": "holiday", "details": "Closed on public holidays"},\n'
        '    {"type": "holiday", "details": "Closed on New Year\'s Day"},\n'
        '    {"type": "holiday", "details": "Closed on Lunar New Year and Chuseok"},\n'
        '    {"type": "entry_closing_time", "details": "Entry closes at 17:30 every day"}\n'
        "  ]\n"
        "}"
    )

    
    response = openai.ChatCompletion.create(
        # model="gpt-3.5-turbo-0125",
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0
    )
    
    return response["choices"][0]["message"]["content"]

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

transformed_data = []

for item in tqdm(data):
    try:
        input_text = item.get("transformed_data", "")
        transformed_json = transform_data_with_gpt(input_text)  
        transformed_data.append({
            "row_index": item["row_index"],
            "transformed_data": json.loads(transformed_json)  
        })
    except Exception as e:
        print(f"Error processing row_index {item['row_index']}: {e}")

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(transformed_data, f, ensure_ascii=False, indent=4)

print(f"Transformed data saved to {output_file}")


  0%|          | 0/5075 [00:00<?, ?it/s]

 67%|██████▋   | 3382/5075 [2:14:05<52:06,  1.85s/it]  

Error processing row_index 8381: Expecting ',' delimiter: line 14 column 2 (char 441)


100%|██████████| 5075/5075 [3:25:23<00:00,  2.43s/it]   


Transformed data saved to transformed_tourapi_shopping_available_time_results_v2_3.json


In [9]:
import json
import pandas as pd

file_path1 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_shopping_available_time_results_v2_1.json"
file_path2 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_shopping_available_time_results_v2_2.json"
file_path3 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_shopping_available_time_results_v2_3.json"


with open(file_path1, 'r', encoding='utf-8') as f1:
    at1 = json.load(f1)

with open(file_path2, 'r', encoding='utf-8') as f2:
    at2 = json.load(f2)

with open(file_path2, 'r', encoding='utf-8') as f3:
    at3 = json.load(f3)

available_time1 = pd.DataFrame(at1)
available_time2 = pd.DataFrame(at2)
available_time3 = pd.DataFrame(at3)

tmp = pd.concat([available_time1, available_time2, available_time3], axis=0)

tourapi.reset_index(drop=True, inplace=True)
tmp.reset_index(drop=True, inplace=True)

tourapi = pd.concat([tourapi, tmp], axis=1)

tourapi.rename(columns={'transformed_data': 'available_time_v2'}, inplace=True)

In [10]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,opening_date,store_information,url,toilet,stroller,pet,credit_card,details,row_index,available_time_v2


In [11]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils 
importlib.reload(analysis_utils)

from analysis_utils import get_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# query = "서울특별시 강남구 가로수길 69 엘큐브elcube"

# geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)
# geocode

query = tourapi.iloc[2693]['address']
geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)

tourapi.loc[2693, 'latitude'] = np.float64(geocode['latitude'])
tourapi.loc[2693, 'longitude'] = np.float64(geocode['longitude'])

In [12]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,opening_date,store_information,url,toilet,stroller,pet,credit_card,details,row_index,available_time_v2


In [13]:
tourapi[tourapi['address'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,opening_date,store_information,url,toilet,stroller,pet,credit_card,details,row_index,available_time_v2


In [14]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_reverse_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# latitude = 37.472404
# longitude = 126.634438

# reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)
# reverse_geocode

latitude, longitude = tourapi.loc[569, ['latitude', 'longitude']]
reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)

tourapi.loc[569, 'address'] = reverse_geocode['address'].strip()

In [15]:
tourapi[tourapi['address'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,opening_date,store_information,url,toilet,stroller,pet,credit_card,details,row_index,available_time_v2


In [16]:
tourapi = tourapi.dropna(subset=['address'])

In [17]:
tourapi[tourapi['description'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,opening_date,store_information,url,toilet,stroller,pet,credit_card,details,row_index,available_time_v2
2797,tourAPI,shopping,카카오프렌즈 플래그십 강남,"서울특별시 서초구 강남대로 429 (서초동,동일빌딩)1,2,3층",37.500846,127.025788,매일 10:30~22:00,불가능,NaN,6612,...,NaN,환급서비스 제공방식 : 즉시+사후,NaN,완비,NaN,NaN,가능,NaN,2797.0,"{'hours': {'Monday': [{'start': '10:30', 'end'..."
4187,tourAPI,shopping,올리브영 이태원중앙,서울특별시 용산구 이태원로 180 (이태원동),37.534278,126.994204,10:00-23:00,불가능,NaN,4406,...,NaN,환급서비스 제공방식 : 사후,NaN,NaN,NaN,NaN,가능,NaN,4187.0,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
8217,tourAPI,shopping,선물가게 바나나,제주특별자치도 서귀포시 일주서로 875,33.258064,126.417138,10:00 ~ 21:30,NaN,NaN,63534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4217.0,"{'hours': {'Monday': [{'start': '10:30', 'end'..."


In [18]:
tourapi[tourapi['description'].isnull()]['place_name'].tolist()

['카카오프렌즈 플래그십 강남', '올리브영 이태원중앙', '선물가게 바나나']

In [19]:
tourapi.loc[tourapi['place_name']=='잠월미술관', 'description'] = "잠월미술관은 전라남도 함평군 해보면 산내리에 위치한 작은 미술관으로, 부부 미술교사인 김광옥과 임혜숙 교사가 퇴직금을 담보로 지었다."
tourapi.loc[tourapi['place_name']=='청주랜드 어린이회관', 'description'] = "청주랜드 어린이회관은 충청북도 청주시에 위치한 가족과 어린이를 위한 복합문화 체험시설이다. 청주랜드 어린이회관은 충청북도 청주시에 위치한 가족과 어린이를 위한 복합문화 체험시설이다. 청주랜드 어린이회관 내 제2관은 청주랜드 어린이체험관으로 변신하였다. 청주랜드 어린이회관은 아동친화 도시를 향한 새로운 패러다임의 가족문화 체험장소이다. 청주랜드 어린이회관 옆 주차장에 주차를 한 후 청주랜드로 입구로 가면 폐품을 이용해 만든 정크아트작품을 만날 수 있다. 청주랜드 어린이회관은 청주동물원, 기후변화체험교육관, 놀이동산 등과 가까운 위치에 있다. "
tourapi.loc[tourapi['place_name']=='동구인문학당', 'description'] = "동구 인문학당은 광주 동구 동명동에 위치한 복합문화공간으로, 지역민들의 인문 배움터이자 문화 향유공간으로 큰 사랑을 받고 있다. 본채, 인문관, 공유 부엌 등 3개 건물과 연못이 있는 마당으로 이뤄져 있다. 본채, 인문관, 공유 부엌 등 3개 건물과 연못이 있는 마당으로 이뤄져 있다. 인문도시를 지향하는 동구가 다양한 인문 프로그램을 진행하기 위해 조성한 공간이다."
tourapi.loc[tourapi['place_name']=='부산예술회관', 'description'] = "부산예술회관은 지하1층 지상4층으로 연면적 4.752m² 규모로써 객석 240석의 소공연장, 전시실2개, 연습실3개소, 부산예총12개 단체의 사무실, 야외공연장, 그리고 주차장, 편의시설 등을 갖추고 있다."
tourapi.loc[tourapi['place_name']=='송파구립 예송미술관', 'description'] = "예송미술관은 송파구민의 시각예술향유를 위한 전시관으로 다양한 프로그램을 운영하는 미술관이다. 예송미술관에서 이번 기간은 4명의 청년 작가의 작품으로 좀 더 깊이있는 시간을 만들 수 있다."
tourapi.loc[tourapi['place_name']=='결성농요농사박물관', 'description'] = "결성농요농사박물관은 홍성군 결성면에 위치한 농업박물관으로, 농사유물과 선사유물 등을 전시하고 있다. 농경생활의 변천사를 한눈에 볼 수 있도록 조성되어 있으며, 농업문화 발굴과 보존을 목표로 하고 있다."
tourapi.loc[tourapi['place_name']=='김포독립운동기념관', 'description'] = "2013년 3월 1일 양촌읍에 개관한 김포시 독립운동기념관은 건축 연면적 2058㎡(623평)에 지상 2층 규모로 건립되었으며 이 지역에서 일어난 독립운동을 재현한 영상과 독립운동을 주도한 인물들을 소개하는 전시관, 일제의 고문실태, 만세운동 체험관 등이 들어서 있다."
tourapi.loc[tourapi['place_name']=='여수자동차극장', 'description'] = "대형 스크린으로 프라이빗한 영화 관람을 즐길 수 있는 자동차 극장이다."

In [20]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_wikipedia_summary

tourist_spot = "잠월미술관"
summary = get_wikipedia_summary(tourist_spot)


In [21]:
tourapi[tourapi['parking'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,opening_date,store_information,url,toilet,stroller,pet,credit_card,details,row_index,available_time_v2
1,tourAPI,shopping,광장시장 한복매장,서울특별시 종로구 창경궁로 88(예지동),37.570065,126.998908,09:00~18:00(점포마다 상이함),NaN,현대화를 시도 하고 있는 다른 재래시장들과는 달리 광장시장은 오래된 역사와 한국 전...,3195,...,1904년,"1층 : 식자재, 이불, 승복지, 원단, 잡화 등<br/>\n2층 : 한복, 포목,...",NaN,있음,불가,NaN,가능,NaN,1.0,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
2,tourAPI,shopping,동대문종합시장 한복상가,서울특별시 종로구 종로 266(종로6가),37.570729,127.007329,08:00 ~ 18:00,NaN,단일 시장으로 동양 최대 규모인 동대문종합시장 내에 위치한 한복 매장은 우리나라 전...,3198,...,1970년 12월23일,"지하 1층 : 원사(실), 커튼 <br />\n1층 : 침구류, 커튼 <br />\...",NaN,있음,불가,NaN,가능,"세금환급방법:현금영수증, 신용카드\n구입방법안내:직접 방문구입 및 홈페이지에 기재된...",2.0,"{'hours': {'Monday': [{'start': '08:00', 'end'..."
3,tourAPI,shopping,"생초시장 (3, 8일)",경상남도 산청군 생초면 어서리 259 생초시장,35.492632,127.835687,06:00 ~ 10:00 이전 (점포별상이),NaN,옛날 시골장 정취를 이제는 느끼기 어렵지만 아직도 시골 아낙네나 할머니들이 남새밭에...,52203,...,NaN,NaN,NaN,NaN,없음,NaN,없음,NaN,3.0,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
4,tourAPI,shopping,원주중원전통시장,강원특별자치도 원주시 중앙시장길 20-3(중앙동),37.350514,127.949600,04:00~21:00,NaN,중원전통시장은 강원도 원주시 중앙동 도심상권에 있는 전통시장이다. 인근에는 중앙시장...,26428,...,NaN,NaN,NaN,NaN,없음,NaN,없음,NaN,4.0,"{'hours': {'Monday': [{'start': '04:00', 'end'..."
18,tourAPI,shopping,"임계사통팔달시장 / 임계장 (5, 10일)",강원특별자치도 정선군 임계면 송계2길 6,37.495014,128.850959,NaN,NaN,수도권에서 동해나 삼척 방면으로 여행을 가다가 영동고속도로 대관령 구간부터 막히기 ...,26110,...,NaN,NaN,NaN,NaN,없음,NaN,없음,NaN,18.0,"{'hours': {'Monday': [], 'Tuesday': [], 'Wedne..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10069,tourAPI,shopping,랩포터리,서울특별시 중구 쌍림동243-1,37.562752,127.005916,10:00~21:00,NaN,랩포터리는 누구나 흙과 친해질 수 있는 공간이다. ‘도자기 실험실’이라는 이름 풀이...,4615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강습안내:성인원데이클래스(물레or핸드빌딩) / 어린이원데이클래스(물레or핸드빌딩) ...,NaN,NaN
10070,tourAPI,shopping,설악눈메골시장,경기도 가평군 신천중앙로 104-1,37.677437,127.492199,10:00~22:00,NaN,"설악눈메골시장은 1942년 5일장(장날 1, 6일)로 개장되어 2020년 8월에 설...",12465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10071,tourAPI,shopping,청평여울시장,경기도 가평군 시장중앙로 19 청평5일시장,37.734632,127.415585,10:00~22:00,NaN,청평은 8~90년대 강변가요제의 추억이 가득한 곳이다. 청평여울시장은 1923년부터...,12453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10072,tourAPI,shopping,KCDF 갤러리숍 공예정원,서울특별시 종로구 인사동11길 8 공예종합유통지원센터KCDF갤러리 1층,37.574062,126.984143,10:00~19:00,NaN,인사동 한국공예·디자인문화진흥원(KCDF) 갤러리 내에 위치한 갤러리숍은 명실 공히...,3145,...,2006년 11월,NaN,NaN,있음,불가,불가,가능,NaN,NaN,NaN


In [22]:
import pandas as pd

parking = pd.read_excel("/home/hwang/projects/yanolja/data/dataset/etc/전국주차장정보표준데이터-20241210.xls")
parking.columns = parking.iloc[0]
parking = parking[1:].reset_index(drop=True)
parking['위도'] = pd.to_numeric(parking['위도'], errors="coerce")
parking['경도'] = pd.to_numeric(parking['경도'], errors="coerce")
parking = parking.dropna(subset=['위도', '경도'])

In [23]:
parking.head(2)

,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,...,결제방법,특기사항,관리기관명,전화번호,위도,경도,장애인전용주차구역보유여부,데이터기준일자,제공기관코드,제공기관명
0,204-2-000023,아래숯골,공영,노외,경기도 성남시 수정구 성남대로 1259,NaN,150,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.442288,127.126662,Y,2024-05-31,3780000,경기도 성남시
1,204-2-000024,태평4동,공영,노외,경기도 성남시 수정구 태평로 109,NaN,89,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.451234,127.138580,Y,2024-05-31,3780000,경기도 성남시


In [24]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import find_nearest_parking

nearest_parking = find_nearest_parking(poi_lat=37.610198, poi_lon=127.040989, parking_data=parking)

In [25]:
nearest_parking

'동방고개 공영 주차장'

In [26]:
import pandas as pd
import numpy as np

def combine_columns(row):
    additional_info = []

    # store_information 추가
    if pd.notna(row['store_information']):
       additional_info.append(f"매장 정보: {row['store_information']}")
    
    # stroller 추가
    if pd.notna(row['stroller']):
        additional_info.append(f"유모차: {row['stroller']}")
    
    # pet 추가
    if pd.notna(row['pet']):
        additional_info.append(f"반려동물: {row['pet']}")
    
    details = row['details'] if pd.notna(row['details']) else ""
    
    combined_details = details + ('\n' if details and additional_info else '') + '\n'.join(additional_info)
    return combined_details

tourapi['details'] = tourapi.apply(combine_columns, axis=1)

tourapi = tourapi.drop(columns=['store_information', 'stroller', 'pet'])


In [27]:
tourapi['source'] = 'tourAPI'
tourapi['category'] = 'shopping'

In [28]:
tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'available_time_v2', 'parking', 'description', 'details']]

In [29]:
tourapi

,source,category,place_name,address,latitude,longitude,available_time,available_time_v2,parking,description,details
0,tourAPI,shopping,동문재래시장,제주특별자치도 제주시 관덕로14길 20,33.511562,126.526059,08:00~21:00,"{'hours': {'Monday': [], 'Tuesday': [{'start':...",가능,동문재래시장은 오랜 역사를 지닌 전통 재래시장으로 제주도를 대표하는 시장 중 하나...,유모차: 없음
1,tourAPI,shopping,광장시장 한복매장,서울특별시 종로구 창경궁로 88(예지동),37.570065,126.998908,09:00~18:00(점포마다 상이함),"{'hours': {'Monday': [], 'Tuesday': [{'start':...",NaN,현대화를 시도 하고 있는 다른 재래시장들과는 달리 광장시장은 오래된 역사와 한국 전...,"매장 정보: 1층 : 식자재, 이불, 승복지, 원단, 잡화 등<br/>\n2층 : ..."
2,tourAPI,shopping,동대문종합시장 한복상가,서울특별시 종로구 종로 266(종로6가),37.570729,127.007329,08:00 ~ 18:00,"{'hours': {'Monday': [{'start': '08:00', 'end'...",NaN,단일 시장으로 동양 최대 규모인 동대문종합시장 내에 위치한 한복 매장은 우리나라 전...,"세금환급방법:현금영수증, 신용카드\n구입방법안내:직접 방문구입 및 홈페이지에 기재된..."
3,tourAPI,shopping,"생초시장 (3, 8일)",경상남도 산청군 생초면 어서리 259 생초시장,35.492632,127.835687,06:00 ~ 10:00 이전 (점포별상이),"{'hours': {'Monday': [], 'Tuesday': [{'start':...",NaN,옛날 시골장 정취를 이제는 느끼기 어렵지만 아직도 시골 아낙네나 할머니들이 남새밭에...,유모차: 없음
4,tourAPI,shopping,원주중원전통시장,강원특별자치도 원주시 중앙시장길 20-3(중앙동),37.350514,127.949600,04:00~21:00,"{'hours': {'Monday': [{'start': '04:00', 'end'...",NaN,중원전통시장은 강원도 원주시 중앙동 도심상권에 있는 전통시장이다. 인근에는 중앙시장...,유모차: 없음
...,...,...,...,...,...,...,...,...,...,...,...
10070,tourAPI,shopping,설악눈메골시장,경기도 가평군 신천중앙로 104-1,37.677437,127.492199,10:00~22:00,NaN,NaN,"설악눈메골시장은 1942년 5일장(장날 1, 6일)로 개장되어 2020년 8월에 설...",
10071,tourAPI,shopping,청평여울시장,경기도 가평군 시장중앙로 19 청평5일시장,37.734632,127.415585,10:00~22:00,NaN,NaN,청평은 8~90년대 강변가요제의 추억이 가득한 곳이다. 청평여울시장은 1923년부터...,
10072,tourAPI,shopping,KCDF 갤러리숍 공예정원,서울특별시 종로구 인사동11길 8 공예종합유통지원센터KCDF갤러리 1층,37.574062,126.984143,10:00~19:00,NaN,NaN,인사동 한국공예·디자인문화진흥원(KCDF) 갤러리 내에 위치한 갤러리숍은 명실 공히...,유모차: 불가\n반려동물: 불가
10073,tourAPI,shopping,코오롱스포츠 (문정직영점),서울특별시 송파구 동남로 128,37.490702,127.125326,10:30 ~ 20:00,NaN,주차 가능,"코오롱스포츠 (문정직영점)은 서울 동남권 최대 규모의 아웃도어, 캠핑 직영점이다. ...",유모차: 불가\n반려동물: 불가
